In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1o7nuc2n
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1o7nuc2n
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=5d8156b34e3fe888f304b3fe6641ff9ef46965712e643ffb27f4e7b706b2b377
  Stored in directory: /tmp/pip-ephem-wheel-cache-ax94l05_/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

[Click here to access the graphs used in the code](https://drive.google.com/drive/folders/1jSrdVKWd6Dw62o_pPZv0nkPcNoBhW_C3?usp=sharing)

In [ ]:
%%cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#define nol 120
#define noi 60000
#define non 1024

typedef struct CudaNoda{
  int id;
  int layer;
  int nin;
  bool isSensor;
  int* inNodes;
  float* inWeights;
  int* dNodes;
  float* dWeights;
}CudaNoda;











__global__ void CUDA_ACTIVATION(CudaNoda* n,float* iv,float* ov,int cl){
  int id = threadIdx.x ;
  if(id==0)printf("n1 %d\n",cl);
  float sum;
  CudaNoda cn;
  cn = n[cl*1024+id];
  if (cn.isSensor ==  1){
      if (iv[id]< 0.0) ov[id] = 0.0;
      else ov[id] = iv[id];
    }
  else{
    sum = -0.3;
    for (int i = 0; i < cn.nin; i++) sum += cn.inWeights[i] * iv[cn.inNodes[i]%1024];
  
  if (sum < 0.0)ov[id] = 0.0;
  else ov[id] = sum;
  }
  if(id==0)printf("n2 %d\n",cl);
}











int main(){
  FILE* fp;
  fp = fopen("drive/MyDrive/0Pt/sparse-images-1024.tsv", "r");
  
  char buffer[100];
  int column = 0;
  
  float* data;
  data = (float*)malloc(sizeof(int) *60000*non);
 
	while (fgets(buffer,100, fp)) 
 	{
		column = 0;
        
		char* value = strtok(buffer, "\t");
    int i;
    int j;
    int val;
		
		while ( value ) {
			if (column == 0) {
        sscanf(value, "%d", &i);
        value = strtok(NULL, "\t");
        column++;
        i--;
			}

			if (column == 1) {
        sscanf(value, "%d", &j);
        value = strtok(NULL, "\t");
        column++;
        j--;
			}

			if (column == 2) {
        sscanf(value, "%d", &val);
        value = strtok(NULL, "\t");
        column++;
			}
		}
   data[i*non+j]=1.;
	}

  fclose(fp);




















  CudaNoda* n;
  n = (CudaNoda*)malloc((nol+1)*non * sizeof(CudaNoda));
  for(int i=0;i<(nol+1);i++){
      for(int j=0;j<non;j++){
          int id=i*non+j;
          n[id].id=id;
          n[id].layer=i;
          n[id].nin=0;
          if (i==0)n[id].isSensor=1;
          else n[id].isSensor=0;
          n[id].inNodes=(int*)malloc(sizeof(int) *non);
          n[id].inWeights=(float*)malloc(sizeof(float) *non);
      }
  }



















  for(int l=1;l<(nol+1);l++){
    char s[100];
    sprintf(s,"drive/MyDrive/0Pt/neuron1024/n1024-l%d.tsv",(l));
    fp = fopen(s, "r");

    char buffer[100];
    int column;
  
    while (fgets(buffer,100, fp)) 
    {
      column = 0;
          
      char* value = strtok(buffer, "\t");
      int i;
      int j;
      int val;
      
      while ( value ) {
        if (column == 0) {
          sscanf(value, "%d", &i);
          value = strtok(NULL, "\t");
          column++;
          i--;
        }

        if (column == 1) {
          sscanf(value, "%d", &j);
          value = strtok(NULL, "\t");
          column++;
          j--;
        }

        if (column == 2) {
          sscanf(value, "%d", &val);
          value = strtok(NULL, "\t");
          column++;
        }
      }
     n[l*non+j].inNodes[n[l*non+j].nin]=(l-1)*non+i;
     n[l*non+j].inWeights[n[l*non+j].nin]=0.065;
     n[l*non+j].nin++;
    }
  fclose(fp);
  }
















for(int i=0;i<(nol+1)*non;i++){
    int *xn;
    int *yn;
    float *xw;
    float *yw;
    yn=n[i].inNodes;
    yw=n[i].inWeights;
    cudaMalloc((void **) &xn, non * sizeof(int));
    cudaMalloc((void **) &xw, non * sizeof(float));
    cudaMemcpy(xn,yn,sizeof(int)*non,cudaMemcpyHostToDevice);
    cudaMemcpy(xw,yw,sizeof(float)*non,cudaMemcpyHostToDevice);
    cudaMemcpy(&(n[i].dNodes),&xn,sizeof(int)*non,cudaMemcpyHostToDevice);
    cudaMemcpy(&(n[i].dWeights),&xw,sizeof(float)*non,cudaMemcpyHostToDevice);
}















  float* iv;
  float* ov;

  CudaNoda* dn;
  float* div;
  float* dov;
  iv = (float*)malloc(sizeof(float)*non);
  ov = (float*)malloc(sizeof(float)*non);
  int sum;
  cudaMalloc(&div,sizeof(float)*non);
  cudaMalloc(&dov,sizeof(float)*non);
  cudaMalloc(&dn,sizeof(CudaNoda)*(nol+1)*non);
  cudaMemcpy(dn,n,sizeof(CudaNoda)*(nol+1)*non,cudaMemcpyHostToDevice);











  struct timeval begin, end;
  gettimeofday(&begin, 0);












  for(int i=0;i<noi;i++){
    for(int j=0;j<non;j++)iv[j]=data[i*non+j];
    for(int k=0;k<(nol+1);k++){
      cudaMemcpy(div,iv,sizeof(float)*non,cudaMemcpyHostToDevice);
      CUDA_ACTIVATION<<<1,non>>>(dn,div,dov,k);
      cudaMemcpy(ov,dov,sizeof(float)*non,cudaMemcpyDeviceToHost);
      for(int j=0;j<non;j++)iv[j]=ov[j];
      sum=0;
      for(int j=0;j<non;j++)if(iv[j]>0)sum++;
    }
    //printf("%d %d\n", i, sum);
  }

















  gettimeofday(&end, 0);
  long seconds = end.tv_sec - begin.tv_sec;
  long microseconds = end.tv_usec - begin.tv_usec;
  double elapsed = seconds + microseconds*1e-6;

  printf("Time measured: %.6f seconds.\n", elapsed);

}

Streaming output truncated to the last 5000 lines.
n1 120
n2 120
n1 0
n2 0
n1 1
n2 1
n1 2
n2 2
n1 3
n2 3
n1 4
n2 4
n1 5
n2 5
n1 6
n2 6
n1 7
n2 7
n1 8
n2 8
n1 9
n2 9
n1 10
n2 10
n1 11
n2 11
n1 12
n2 12
n1 13
n2 13
n1 14
n2 14
n1 15
n2 15
n1 16
n2 16
n1 17
n2 17
n1 18
n2 18
n1 19
n2 19
n1 20
n2 20
n1 21
n2 21
n1 22
n2 22
n1 23
n2 23
n1 24
n2 24
n1 25
n2 25
n1 26
n2 26
n1 27
n2 27
n1 28
n2 28
n1 29
n2 29
n1 30
n2 30
n1 31
n2 31
n1 32
n2 32
n1 33
n2 33
n1 34
n2 34
n1 35
n2 35
n1 36
n2 36
n1 37
n2 37
n1 38
n2 38
n1 39
n2 39
n1 40
n2 40
n1 41
n2 41
n1 42
n2 42
n1 43
n2 43
n1 44
n2 44
n1 45
n2 45
n1 46
n2 46
n1 47
n2 47
n1 48
n2 48
n1 49
n2 49
n1 50
n2 50
n1 51
n2 51
n1 52
n2 52
n1 53
n2 53
n1 54
n2 54
n1 55
n2 55
n1 56
n2 56
n1 57
n2 57
n1 58
n2 58
n1 59
n2 59
n1 60
n2 60
n1 61
n2 61
n1 62
n2 62
n1 63
n2 63
n1 64
n2 64
n1 65
n2 65
n1 66
n2 66
n1 67
n2 67
n1 68
n2 68
n1 69
n2 69
n1 70
n2 70
n1 71
n2 71
n1 72
n2 72
n1 73
n2 73
n1 74
n2 74
n1 75
n2 75
n1 76
n2 76
n1 77
n2 77
n1 78
n2 78
n1 79
n